# Notebook Overview

This notebook is designed to facilitate the exploration and management of metadata for various relational databases within a Databricks environment. It provides a unified interface for connecting to different RDBMS systems, retrieving metadata about available tables and columns, and managing connection details.

## Key Features

- **Connection Management:**  
  Users can specify connection details (such as connection ID, schema, target database, and storage options) using Databricks widgets. The notebook supports multiple RDBMS types, including Oracle, SQL Server, PostgreSQL, MySQL, MariaDB, Snowflake, BigQuery, Redshift, DB2, and SAP HANA.

- **Metadata Retrieval:**  
  The notebook loads and queries metadata tables (`connection_metadata` and `table_metadata`) to provide information about available connections and their associated tables.

- **Dynamic JDBC Configuration:**  
  Based on the selected connection type, the notebook dynamically constructs the appropriate JDBC URL, driver, and SQL queries to retrieve table and column metadata.

- **Automated Dependency Management:**  
  The notebook automatically installs the required Python database drivers using `%pip` based on the selected RDBMS type.

- **Extensible MetadataManager Class:**  
  The `MetadataManager` class encapsulates logic for loading metadata tables, making it easy to extend or modify metadata operations.

## Workflow

1. **User Input:**  
   Users provide connection and target details via widgets.

2. **Connection Validation:**  
   The notebook checks if metadata for the selected connection already exists to prevent duplication.

3. **Metadata Loading:**  
   Metadata for connections and tables is loaded using the `MetadataManager` class.

4. **JDBC and Query Preparation:**  
   The notebook constructs the correct JDBC URL, driver, and metadata queries for the selected RDBMS.

5. **Column Metadata Retrieval:**  
   The notebook provides a function to generate column metadata queries for each supported RDBMS.

## Usage

- Use the widgets at the top of the notebook to specify your connection and target parameters.
- The notebook will automatically handle driver installation and metadata retrieval based on your selections.
- You can extend the notebook to perform further data exploration, ETL, or analytics tasks as needed.

---

This notebook is intended for data engineers and analysts who need to manage and explore metadata across heterogeneous data sources in a Databricks environment.

✅ If target_path_type is anything else (default local mount):
The system will default to a general path under /mnt/datalake.

You must provide:

tgt_tbl_name: The target table or file name.

(Optional) target_db: Used as a folder inside /mnt/datalake.

Resulting path example:
/mnt/datalake/<target_db>/<tgt_tbl_name>

✅ If target_path_type is abfss (Azure Data Lake Gen2):
This format is used for writing to Azure Storage accounts with hierarchical namespaces.

You must provide:

target_container: The container name inside your Azure storage account.

target_account: The Azure storage account name.

tgt_tbl_name: The target table or file name.

(Optional) target_db: If provided, it will be used as a folder inside the container.

Resulting path example:
abfss://<target_container>@<target_account>.dfs.core.windows.net/<target_db>/<tgt_tbl_name>

✅ If target_path_type is s3 (Amazon S3):
This format is used for writing to AWS S3 buckets.

You must provide:

target_bucket: Your S3 bucket name.

tgt_tbl_name: The target table or file name.

(Optional) target_db: If provided, it will be used as a folder in the bucket.

Resulting path example:
s3://<target_bucket>/<target_db>/<tgt_tbl_name>

✅ If target_path_type is dbfs (Databricks File System):
This is used to write files under a mounted path in Databricks File System.

You must provide:

target_mount: The DBFS mount point.

tgt_tbl_name: The target table or file name.

(Optional) target_db: Used as a folder inside the mount.

Resulting path example:
/dbfs/mnt/<target_mount>/<target_db>/<tgt_tbl_name>



In [0]:
# Widgets for connection_id, schema_name, target_db, prefix, suffix, path_type, and storage details
dbutils.widgets.text("connection_id", "", "Connection ID")
dbutils.widgets.text("schema_name", "", "Schema Name")
dbutils.widgets.text("target_db", "", "Target Databricks,abfss,s3,dbfs Database")
dbutils.widgets.text("target_prefix", "", "Target Table Prefix (optional)")
dbutils.widgets.text("target_suffix", "", "Target Table Suffix (optional)")
dbutils.widgets.text("target_path_type", "", "Target Path Type (optional)")
dbutils.widgets.text("target_container", "", "Target Container (for abfss)")
dbutils.widgets.text("target_account", "", "Target Account (for abfss)")
dbutils.widgets.text("target_bucket", "", "Target Bucket (for s3)")
dbutils.widgets.text("target_mount", "", "Target Mount (for dbfs)")

connection_id = dbutils.widgets.get("connection_id")
schema_name = dbutils.widgets.get("schema_name")
target_db = dbutils.widgets.get("target_db")
target_prefix = dbutils.widgets.get("target_prefix")
target_suffix = dbutils.widgets.get("target_suffix")
target_path_type = dbutils.widgets.get("target_path_type")
target_container = dbutils.widgets.get("target_container")
target_account = dbutils.widgets.get("target_account")
target_bucket = dbutils.widgets.get("target_bucket")
target_mount = dbutils.widgets.get("target_mount")

In [0]:
if spark.sql(f"SELECT 1 FROM workspace.default.table_metadata WHERE connection_id = '{connection_id}' LIMIT 1").count() != 0:
    dbutils.notebook.exit("Metadata already exist")

In [0]:
from pyspark.sql import SparkSession, Row

In [0]:
class MetadataManager:
    def __init__(self, spark: SparkSession):
        self.spark = spark

    def load_connection_metadata(self):
        return self.spark.table('connection_metadata')

    def load_table_metadata(self):
        return self.spark.table('table_metadata')

In [0]:
spark = SparkSession.builder.getOrCreate()
metadata = MetadataManager(spark)
conn_meta = metadata.load_connection_metadata()

In [0]:
#connection_id='conn_postgres_netflix'
target_prefix='stg_'
target_db='default'

In [0]:
conn = conn_meta.filter(conn_meta.connection_id == connection_id).collect()[0]
print(conn)

In [0]:
if conn.type == "oracle":
    %pip install cx_Oracle
elif conn.type == "sqlserver":
    %pip install pyodbc
elif conn.type == "postgresql":
    %pip install psycopg2-binary
elif conn.type == "mysql":
    %pip install mysql-connector-python
elif conn.type == "mariadb":
    %pip install mariadb
elif conn.type == "snowflake":
    %pip install snowflake-connector-python
elif conn.type == "bigquery":
    %pip install google-cloud-bigquery
elif conn.type == "redshift":
    %pip install redshift-connector
elif conn.type == "db2":
    %pip install ibm-db
elif conn.type == "hana":
    %pip install hdbcli
else:
    raise Exception(f"Unsupported RDBMS type: {conn.type}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 20.0 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
jdbc_url = 'None'
table_query = 'None'
connection_properties = {
    "user": conn.username,
    "password": conn.password,
    "driver": 'None'
}
if conn.type == "oracle":
    jdbc_url = f"jdbc:oracle:thin:@{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT table_name, "
        f"'{conn.schema.upper()}.' || table_name AS table_call_name "
        f"FROM all_tables WHERE owner = '{conn.schema.upper()}') AS t"
    )
    connection_properties["driver"] = "oracle.jdbc.OracleDriver"
elif conn.type == "sqlserver":
    jdbc_url = f"jdbc:sqlserver://{conn.host}:{conn.port};databaseName={conn.database}"
    table_query = (
        f"(SELECT TABLE_NAME as table_name, "
        f"'[{conn.schema}].[' + TABLE_NAME + ']' AS table_call_name "
        f"FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = '{conn.schema}') AS t"
    )
    connection_properties["driver"] = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
elif conn.type == "postgresql":
    jdbc_url = f"jdbc:postgresql://{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT table_name, "
        f"'\"{conn.schema}\".\"' || table_name || '\"' AS table_call_name "
        f"FROM information_schema.tables WHERE table_schema = '{conn.schema}') AS t"
    )
    connection_properties["driver"] = "org.postgresql.Driver"
elif conn.type == "mysql":
    jdbc_url = f"jdbc:mysql://{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT table_name, "
        f"CONCAT('`{conn.schema}`.`', table_name, '`') AS table_call_name "
        f"FROM information_schema.tables WHERE table_schema = '{conn.schema}') AS t"
    )
    connection_properties["driver"] = "com.mysql.cj.jdbc.Driver"
elif conn.type == "mariadb":
    jdbc_url = f"jdbc:mariadb://{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT table_name, "
        f"CONCAT('`{conn.schema}`.`', table_name, '`') AS table_call_name "
        f"FROM information_schema.tables WHERE table_schema = '{conn.schema}') AS t"
    )
    connection_properties["driver"] = "org.mariadb.jdbc.Driver"
elif conn.type == "snowflake":
    jdbc_url = f"jdbc:snowflake://{conn.host}/?db={conn.database}&schema={conn.schema}"
    table_query = (
        f"(SELECT table_name, "
        f"'\"{conn.schema.upper()}\".\"' || table_name || '\"' AS table_call_name "
        f"FROM information_schema.tables WHERE table_schema = '{conn.schema.upper()}') AS t"
    )
    connection_properties["driver"] = "net.snowflake.client.jdbc.SnowflakeDriver"
elif conn.type == "bigquery":
    jdbc_url = f"jdbc:bigquery://https://www.googleapis.com/bigquery/v2:443;ProjectId={conn.database};"
    table_query = (
        f"(SELECT table_name, "
        f"CONCAT('`{conn.schema}.', table_name, '`') AS table_call_name "
        f"FROM `{conn.schema}.INFORMATION_SCHEMA.TABLES`) AS t"
    )
    connection_properties["driver"] = "com.simba.googlebigquery.jdbc42.Driver"
elif conn.type == "redshift":
    jdbc_url = f"jdbc:redshift://{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT tablename AS table_name, "
        f"'\"{conn.schema}\".\"' || tablename || '\"' AS table_call_name "
        f"FROM pg_table_def WHERE schemaname = '{conn.schema}') AS t"
    )
    connection_properties["driver"] = "com.amazon.redshift.jdbc.Driver"
elif conn.type == "db2":
    jdbc_url = f"jdbc:db2://{conn.host}:{conn.port}/{conn.database}"
    table_query = (
        f"(SELECT tabname AS table_name, "
        f"'{conn.schema.upper()}.' || tabname AS table_call_name "
        f"FROM syscat.tables WHERE tabschema = '{conn.schema.upper()}') AS t"
    )
    connection_properties["driver"] = "com.ibm.db2.jcc.DB2Driver"
elif conn.type == "hana":
    jdbc_url = f"jdbc:sap://{conn.host}:{conn.port}"
    table_query = (
        f"(SELECT TABLE_NAME, "
        f"'{conn.schema.upper()}.' || TABLE_NAME AS table_call_name "
        f"FROM TABLES WHERE SCHEMA_NAME = '{conn.schema.upper()}') AS t"
    )
    connection_properties["driver"] = "com.sap.db.jdbc.Driver"
else:
    raise Exception(f"Unsupported RDBMS type: {conn.type}")

print(table_query)

(SELECT table_name, CONCAT('`sakila`.`', table_name, '`') AS table_call_name FROM information_schema.tables WHERE table_schema = 'sakila') AS t


In [0]:
def get_column_query(rdbms_type, schema, table_name):
    if rdbms_type == "oracle":
        return f"(SELECT column_name, data_type, nullable FROM all_tab_columns WHERE owner = '{schema.upper()}' AND table_name = '{table_name.upper()}') AS c"
    elif rdbms_type == "sqlserver":
        return f"(SELECT COLUMN_NAME as column_name, DATA_TYPE as data_type, IS_NULLABLE as nullable FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_SCHEMA = '{schema}' AND TABLE_NAME = '{table_name}') AS c"
    elif rdbms_type in ["postgresql", "redshift"]:
        return f"(SELECT column_name, data_type, is_nullable as nullable FROM information_schema.columns WHERE table_schema = '{schema}' AND table_name = '{table_name}') AS c"
    elif rdbms_type in ["mysql", "mariadb"]:
        return f"(SELECT column_name, data_type, is_nullable as nullable FROM information_schema.columns WHERE table_schema = '{schema}' AND table_name = '{table_name}') AS c"
    elif rdbms_type == "snowflake":
        return f"(SELECT column_name, data_type, is_nullable as nullable FROM information_schema.columns WHERE table_schema = '{schema.upper()}' AND table_name = '{table_name.upper()}') AS c"
    elif rdbms_type == "bigquery":
        return f"(SELECT column_name, data_type, is_nullable as nullable FROM `{schema}.INFORMATION_SCHEMA.COLUMNS` WHERE table_name = '{table_name}') AS c"
    elif rdbms_type == "db2":
        return f"(SELECT colname as column_name, typename as data_type, nulls as nullable FROM syscat.columns WHERE tabschema = '{schema.upper()}' AND tabname = '{table_name.upper()}') AS c"
    elif rdbms_type == "hana":
        return f"(SELECT COLUMN_NAME as column_name, DATA_TYPE_NAME as data_type, IS_NULLABLE as nullable FROM TABLE_COLUMNS WHERE SCHEMA_NAME = '{schema.upper()}' AND TABLE_NAME = '{table_name.upper()}') AS c"
    else:
        raise Exception(f"Unsupported RDBMS type: {rdbms_type}")

In [0]:
type_map = {
    # Oracle
    "NUMBER": "long",
    "VARCHAR2": "string",
    "DATE": "timestamp",
    "CHAR": "string",
    "NVARCHAR2": "string",
    "FLOAT": "double",
    # SQL Server
    "int": "int",
    "bigint": "long",
    "nvarchar": "string",
    "varchar": "string",
    "datetime": "timestamp",
    "bit": "boolean",
    # PostgreSQL
    "integer": "int",
    "serial": "int",
    "bigserial": "long",
    "text": "string",
    "timestamp": "timestamp",
    # MySQL/MariaDB
    "tinyint": "byte",
    "smallint": "short",
    "mediumint": "int",
    "decimal": "double",
    # Snowflake/Redshift/BigQuery/Db2/HANA (add more as needed)
    "BOOLEAN": "boolean",
    "STRING": "string",
    "FLOAT64": "double",
    "TIMESTAMP": "timestamp",
    "DATE": "date"
}


In [0]:
%pip install mysql-connector-python

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
# Fetch tables from source schema

try:
    tables_df = spark.read.jdbc(
        url=jdbc_url,
        table=table_query,
        properties=connection_properties
    )
except Exception as e:
    print(f"Error loading tables: {e}")

table_names = [(row.table_name, row.table_call_name) for row in tables_df.collect()]
display(table_names)

---------------------------------------------------------------------------
SparkConnectGrpcException                 Traceback (most recent call last)
File <command-5108592265000802>, line 12
      9 except Exception as e:
     10     print(f"Error loading tables: {e}")
---> 12 table_names = [(row.table_name, row.table_call_name) for row in tables_df.collect()]
     13 display(table_names)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1915, in DataFrame.collect(self)
   1914 def collect(self) -> List[Row]:
-> 1915     table, schema = self._to_table()
   1917     # not all datatypes are supported in arrow based collect
   1918     # here always verify the schema by from_arrow_schema
   1919     schema2 = from_arrow_schema(table.schema, prefer_timestamp_ntz=True)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1994, in DataFrame._to_table(self)
   1992 def _to_table(self) -> Tuple["pa.Table", Optional[Struct

In [0]:
# Prepare rows for table_metadata and column_metadata (with sensible defaults)
table_metadata_rows = []
column_metadata_rows = []
for tbl_name, tbl_call_name in table_names:
    table_id = f"{connection_id}_{tbl_name}"
    tgt_tbl_name = f"{target_prefix}{tbl_name}{target_suffix}" if target_prefix or target_suffix else tbl_name

    # Set target_path based on path type, including target_db if specified
    if target_path_type == "abfss":
        if target_db:
            target_path = f"abfss://{target_container}@{target_account}.dfs.core.windows.net/{target_db}/{tgt_tbl_name}"
        else:
            target_path = f"abfss://{target_container}@{target_account}.dfs.core.windows.net/{tgt_tbl_name}"
    elif target_path_type == "s3":
        if target_db:
            target_path = f"s3://{target_bucket}/{target_db}/{tgt_tbl_name}"
        else:
            target_path = f"s3://{target_bucket}/{tgt_tbl_name}"
    elif target_path_type == "dbfs":
        if target_db:
            target_path = f"/dbfs/mnt/{target_mount}/{target_db}/{tgt_tbl_name}"
        else:
            target_path = f"/dbfs/mnt/{target_mount}/{tgt_tbl_name}"
    else:
        if target_db:
            target_path = f"/mnt/datalake/{target_db}/{tgt_tbl_name}"
        else:
            target_path = f"/mnt/datalake/{tgt_tbl_name}"

    table_metadata_rows.append(Row(
        table_id=table_id,
        connection_id=connection_id,
        table_name=tbl_name,
        target_table_name=tgt_tbl_name,
        table_type="full",
        primary_key_columns='None',
        watermark_column='None',
        partition_column='None',
        target_path=target_path,
        load_frequency="daily",
        active_flag="Y",
        comments="Auto-populated",
        optimize_zorder_by='None',
        repartition_columns='None',
        num_output_files='None',
        write_mode="overwrite",
        cache_intermediate='False',
        target_db=target_db,
        table_call_name=tbl_call_name
    ))
    # Fetch columns for this table
    col_query = get_column_query(conn.type, conn.schema, tbl_name)
    
    cols_df = spark.read.jdbc(
        url=jdbc_url,
        table=col_query,
        properties=connection_properties
    )
    for col in cols_df.collect():
        column_metadata_rows.append(Row(
            table_id=table_id,
            column_name=col.column_name,
            data_type=col.data_type,
            target_type=type_map.get(col.data_type.upper(), "string"),
            target_column_name=col.column_name,  # Default to source column name
            nullable=(str(col.nullable).upper() in ["YES", "Y", "NULL", "1"]),
            is_primary_key="N"  # Could be improved with PK detection logic
        ))

In [0]:
auto_df = spark.createDataFrame(table_metadata_rows)

existing_df = spark.table("table_metadata").select("connection_id", "table_id")
auto_df = auto_df.join(
    existing_df,
    on=["connection_id", "table_id"],
    how="left_anti"
)

#display(auto_df)
auto_df.write.format("delta").mode("append").saveAsTable("table_metadata")

In [0]:
col_auto_df = spark.createDataFrame(column_metadata_rows)

existing_col_df = spark.table("column_metadata").select("table_id", "column_name")
col_auto_df = col_auto_df.join(
    existing_col_df,
    on=["table_id", "column_name"],
    how="left_anti"
)

col_auto_df.write.format("delta").mode("append").saveAsTable("column_metadata")